In [1]:
!pip install portfoliolab

     |████████████████████████████████| 104 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 30.7 MB 1.0 MB/s eta 0:00:01
  Using cached pandas-1.1.5-cp38-cp38-macosx_10_9_x86_64.whl (10.1 MB)
     |████████████████████████████████| 12.5 MB 1.2 MB/s eta 0:00:01    |███████▍                        | 2.9 MB 391 kB/s eta 0:00:25
     |████████████████████████████████| 16.0 MB 165 kB/s eta 0:00:01
     |████████████████████████████████| 845 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 164 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 2.6 MB/s eta 0:00:01
  Created wheel for ecos: filename=ecos-2.0.7.post1-cp38-cp38-macosx_10_9_x86_64.whl size=83726 sha256=cde4e6a5fb99d9c7aa240b1d47f586b15f232d2908cf3f4644f53e7154a6a825
  Stored in directory: /Users/damian/Library/Caches/pip/wheels/c3/c1/ce/b551b0b3a

In [2]:
from portfoliolab.bayesian import VanillaBlackLitterman

In [4]:
import pandas as pd

In [5]:
countries = ['AU', 'CA', 'FR', 'DE', 'JP', 'UK', 'US']
# Table 1 of the He-Litterman paper: Correlation matrix
correlation = pd.DataFrame([
    [1.000, 0.488, 0.478, 0.515, 0.439, 0.512, 0.491],
    [0.488, 1.000, 0.664, 0.655, 0.310, 0.608, 0.779],
    [0.478, 0.664, 1.000, 0.861, 0.355, 0.783, 0.668],
    [0.515, 0.655, 0.861, 1.000, 0.354, 0.777, 0.653],
    [0.439, 0.310, 0.355, 0.354, 1.000, 0.405, 0.306],
    [0.512, 0.608, 0.783, 0.777, 0.405, 1.000, 0.652],
    [0.491, 0.779, 0.668, 0.653, 0.306, 0.652, 1.000]
], index=countries, columns=countries)
# Table 2 of the He-Litterman paper: Volatilities
volatilities = pd.DataFrame([0.160, 0.203, 0.248, 0.271, 0.210, 0.200, 0.187],
                            index=countries, columns=["vol"])
covariance = volatilities.dot(volatilities.T) * correlation

In [6]:
# Table 2 of the He-Litterman paper: Market-capitalised weights
market_weights = pd.DataFrame([0.016, 0.022, 0.052, 0.055, 0.116, 0.124, 0.615],
                              index=countries, columns=["CapWeight"])

# View 1

In [7]:
# Q
views = [0.05]
# P
pick_list = [
        {
            "DE": 1.0,
            "FR": -market_weights.loc["FR"]/(market_weights.loc["FR"] + \
                                            market_weights.loc["UK"]),
            "UK": -market_weights.loc["UK"] / (market_weights.loc["FR"] + \
                                                market_weights.loc["UK"])
        }
]

In [8]:
bl = VanillaBlackLitterman()
bl.allocate(covariance=covariance,
            market_capitalised_weights=market_weights,
            investor_views=views,
            pick_list=pick_list,
            asset_names=covariance.columns,
            tau=0.05,
            risk_aversion=2.5)

/Users/damian/opt/anaconda3/envs/hands_on_ml2/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [12]:
bl.implied_equilibrium_returns

,AU,CA,FR,DE,JP,UK,US
0,0.039376,0.069152,0.083581,0.090272,0.043028,0.067677,0.0756


In [ ]:
bl.implied_